In [ ]:
import os
from anthropic import Anthropic
os.environ["ANTHROPIC_API_KEY"] = "your_api_key_here" 
#print (ANTHROPIC_API_KEY)
client = Anthropic()

message = client.messages.create(
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": "Hello, Claude",
        }
    ],
    model="claude-3-5-sonnet-20240620",
)
print(message.content)

In [ ]:
import os
from anthropic import Anthropic
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool, DirectoryReadTool, FileReadTool
from crewai_tools import BaseTool
from langchain_anthropic import ChatAnthropic
from langchain_openai import ChatOpenAI

# Set your API keys (replace 'Your Key' with actual keys)
import os
os.environ["SERPER_API_KEY"] = "your_api_key_here" # serper.dev API key

os.environ["OPENAI_API_KEY"] = "your_api_key_here"
os.environ["ANTHROPIC_API_KEY"] = "your_api_key_here" 
#print (ANTHROPIC_API_KEY)
#client = Anthropic()

# Initialize the tools
search_tool = SerperDevTool()
directory_read_tool = DirectoryReadTool(directory='company_docs')
file_read_tool = FileReadTool()



# Define the Validation Agent
validation_agent = Agent(
    role="Validation Specialist",
    goal="Validate and refine the criteria of the job requirements from {job_requirements}.",
    verbose=True,
    memory=True,
    backstory=(
        "You are an expert in ensuring that criteria of job requirements in {job_requirements}, are clear and comprehensive. "
        "Your attention to detail helps identify missing information and refine statements for further analysis."
    ),
    #tools=[search_tool],
    llm=ChatAnthropic(model="claude-3-5-sonnet-20240620", verbose=True),
    allow_delegation=True
)

# Define the Analysis Agent
analysis_agent = Agent(
    role="Analysis Specialist",
    goal='Analyze the refined statement from validation_agent to identify key criteria for the job.',
    verbose=True,
    memory=True,
    backstory=(
        "With years of experience in HR and recruitment, you excel at understanding {job_requirements} to uncover the underlying needs. "
    ),
    #tools=[search_tool],
    llm=ChatOpenAI(model="gpt-3.5-turbo", verbose=True),
    allow_delegation=True
)

# Define the Report Generation Agent (HR Consultant)
report_generation_agent = Agent(
    role="Report Generation Specialist",
    goal="Match the resumes in line with key criteria from from analysis_agent"
         "You will then generate a comprehensive report",
    verbose=True,
    memory=True,
    backstory=(
        "You have a knack for compiling resume match reports."
        "You reports are known for their clarity, depth, and practical recommendations."
    ),
    #tools=[search_tool],
    llm=ChatOpenAI(model="gpt-3.5-turbo", verbose=True),
    allow_delegation=False
)

# Task for validating the job description
validate_problem_task = Task(
    description=(
        "Review the criteria for job in {job_requirements} and ensure all necessary details are provided. "
        "Identify any missing or unclear information and flag it for further review. "
        "Prepare a refined job description that is clear and actionable based on the provided details."
    ),
    expected_output='A validated and refined job description highlighting any missing information.',
    #tools=[search_tool],
    agent=validation_agent
)

# Task for understanding the job description
understand_problem_task = Task(
    description=(
        "Analyze the validated job description using your extensive HR and recruiting experience. "
        "Identify the key criteria, and read between the lines."
        "Prepare the final set of criteria that are needed in prospective candidates"
    ),
    expected_output='An understanding analysis of the job requirement, highlighting key criteria to be matched in prospective candidates.',
    #tools=[search_tool],
    agent=analysis_agent,
    output_file='job_analysis.txt'
)

# Task for conducting match and generating a report
generate_report_task = Task(
    description=(
        "Match the final set of  criteria with the prospective candidates resumes' contents."
        "Provide a detailed report with the overall match percentage, individual criteria match and a brief qualitative summary."
        "If for anyreason you think the candidate is an outlier mention so and give a brief reasoning."
        
    ),
    expected_output= "A nice professional report in markdown format with an understanding of the {job_requirements}, overall match percentage and recommendation y/n for in-person interview, individual criteria match percentage, a qualitative summary, and if outlier, a reasoning.",
    #tools=[search_tool],
    tools=[directory_read_tool, file_read_tool],
    agent=report_generation_agent,
    output_file='sample_report.md'
)


# Define the crew with multiple agents and the tasks
consulting_crew = Crew(
    agents=[validation_agent, analysis_agent, report_generation_agent],
    tasks=[validate_problem_task, understand_problem_task, generate_report_task],
    process=Process.sequential
)

# Sample client problem filled out using the template

# Specify the path to your text file
file_path = 'job_description.txt'

# Open the file in read mode and store its contents in a variable
with open(file_path, 'r') as file:
    file_contents = file.read()


job_requirements= str(file_contents)
print (job_requirements)




In [ ]:
 
# Kickoff the process with the sample client problem statement
result = consulting_crew.kickoff(inputs={'job_requirements': job_requirements})
#result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)
result = consulting_crew.kickoff(inputs={'job_requirements': job_requirements})
#result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})
print(result)

In [ ]:
from IPython.display import Markdown
Markdown(result)